In [1]:
#Use % sign is using jupyter notebook ensure the installation occurs in the environment that the Jupyter notebook kernel is using
#Python Version used for the project : Python 3.10.12 
%pip install ultralytics==8.2.88
%pip install pillow==10.2.0 
%pip install gdown==5.2.0


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
#download the sample images 
!gdown 1wCfC5Jl_cYOOKCOX2GtTf8sdtFgsiCmS 

Downloading...
From (original): https://drive.google.com/uc?id=1wCfC5Jl_cYOOKCOX2GtTf8sdtFgsiCmS
From (redirected): https://drive.google.com/uc?id=1wCfC5Jl_cYOOKCOX2GtTf8sdtFgsiCmS&confirm=t&uuid=6db49a15-acd9-4066-871c-627351c82628
To: /Users/bicharshresthagurung/Documents/Biomedical Department/hbN-crack-detection/sample-images.zip
100%|████████████████████████████████████████| 107M/107M [00:03<00:00, 30.1MB/s]


In [3]:
!unzip -o ./sample-images.zip 

Archive:  ./sample-images.zip
   creating: sample-images/
  inflating: __MACOSX/._sample-images  
  inflating: sample-images/Y2X7_S2.png  
  inflating: __MACOSX/sample-images/._Y2X7_S2.png  
  inflating: sample-images/Y1X3_S9.png  
  inflating: __MACOSX/sample-images/._Y1X3_S9.png  
  inflating: sample-images/Y5X3_S2.png  
  inflating: __MACOSX/sample-images/._Y5X3_S2.png  
  inflating: sample-images/Y6X2_S2.png  
  inflating: __MACOSX/sample-images/._Y6X2_S2.png  
  inflating: sample-images/.DS_Store  
  inflating: __MACOSX/sample-images/._.DS_Store  
  inflating: sample-images/Y4X2_S10.png  
  inflating: __MACOSX/sample-images/._Y4X2_S10.png  
  inflating: sample-images/Y4X6_S8.png  
  inflating: __MACOSX/sample-images/._Y4X6_S8.png  
  inflating: sample-images/Y3X7_S6.png  
  inflating: __MACOSX/sample-images/._Y3X7_S6.png  
  inflating: sample-images/Y2X7_S6.png  
  inflating: __MACOSX/sample-images/._Y2X7_S6.png  
  inflating: sample-images/Y6X4_S4.png  
  inflating: __MACOSX/samp

In [4]:
#download the model weight
!gdown 1-k_3To9pPOmFAj5doHXJig22L9p1QDYP -O weights/

Downloading...
From: https://drive.google.com/uc?id=1-k_3To9pPOmFAj5doHXJig22L9p1QDYP
To: /Users/bicharshresthagurung/Documents/Biomedical Department/hbN-crack-detection/weights/trained_weight.pt
100%|██████████████████████████████████████| 6.23M/6.23M [00:00<00:00, 14.1MB/s]


In [5]:
from ultralytics import YOLO
import os
from PIL import Image
import json

In [6]:

def ensure_dir(directory):
    """Ensure that a directory exists. If it doesn't, create it."""
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")
        
def load_model(model_path):
    """Load the YOLO model from the specified path."""
    return YOLO(model_path)

def get_image_files(image_dir):
    """Retrieve image file paths from the specified directory and validate if they are images."""
    image_files = []
    for f in os.listdir(image_dir):
        file_path = os.path.join(image_dir, f)
        if os.path.isfile(file_path):
            try:
                with Image.open(file_path) as img:
                    img.verify()  # Verify that this is an image
                image_files.append(file_path)
            except (IOError, SyntaxError):
                print(f"Warning: {file_path} is not a valid image and will be skipped.")
    return image_files


def convert_tensor_to_list(tensor):
    """Convert PyTorch tensor to a list for JSON serialization."""
    return tensor.numpy().tolist() if tensor is not None else None
    

def run_inference(model, image_files):
    """Run inference on the list of image files using the loaded model."""
    results = model(image_files)
    return results

def save_results(results, image_files, output_dir):
    """Save and serialize the results of the inference."""
    ensure_dir(output_dir)  # Ensure the output directory exists
    for index, result in enumerate(results):
        base_filename = os.path.basename(image_files[index])
        json_filename = f"{os.path.splitext(base_filename)[0]}.json"
        json_path = os.path.join(output_dir, json_filename)
        
        # Prepare JSON data
        json_data = {
            "filename": base_filename,
            "boxes": convert_tensor_to_list(result.boxes.xyxy),
            "masks": convert_tensor_to_list(result.masks) if result.masks else None,
            "keypoints": convert_tensor_to_list(result.keypoints) if result.keypoints else None,
            "probs": convert_tensor_to_list(result.probs) if result.probs else None,
            "obb": convert_tensor_to_list(result.obb) if result.obb else None,
        }

        # Save JSON data
        with open(json_path, 'w') as jf:
            json.dump(json_data, jf, indent=4)
        print(f"Results saved to JSON: {json_path}")

        # Display and save image results
        result.show()  # display to screen
        result_image_path = os.path.join(output_dir, base_filename)
        result.save(filename=result_image_path)  # save to disk
        print(f"Results image saved to: {result_image_path}")

In [ ]:
def main():
    # Model and directory settings
    model_path = 'weights/trained_weight.pt'
    image_dir = 'sample-images'
    output_dir = "results"
    
    # Load the model
    model = load_model(model_path)
    
    # Get list of image files
    image_files = get_image_files(image_dir)
    
    # Check if image files are found
    if not image_files:
        print("No images found in the directory:", image_dir)
        return
    
    # Run batched inference on images
    results = run_inference(model, image_files)
    
    # Process results list
    save_results(results, image_files, output_dir)

if __name__ == '__main__':
    main()


0: 640x640 6 Cracks, 133.5ms
1: 640x640 (no detections), 133.5ms
2: 640x640 4 Cracks, 133.5ms
3: 640x640 1 Crack, 133.5ms
4: 640x640 4 Cracks, 133.5ms
5: 640x640 3 Cracks, 133.5ms
6: 640x640 4 Cracks, 133.5ms
7: 640x640 1 Crack, 133.5ms
8: 640x640 2 Cracks, 133.5ms
9: 640x640 5 Cracks, 133.5ms
10: 640x640 4 Cracks, 133.5ms
11: 640x640 2 Cracks, 133.5ms
12: 640x640 3 Cracks, 133.5ms
13: 640x640 1 Crack, 133.5ms
14: 640x640 2 Cracks, 133.5ms
15: 640x640 4 Cracks, 133.5ms
16: 640x640 4 Cracks, 133.5ms
17: 640x640 3 Cracks, 133.5ms
18: 640x640 1 Crack, 133.5ms
19: 640x640 1 Crack, 133.5ms
20: 640x640 7 Cracks, 133.5ms
21: 640x640 2 Cracks, 133.5ms
22: 640x640 (no detections), 133.5ms
23: 640x640 3 Cracks, 133.5ms
Speed: 3.6ms preprocess, 133.5ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)
Created directory: results
Results saved to JSON: results/Y2X7_S2.json
Results image saved to: results/Y2X7_S2.png
Results saved to JSON: results/Y1X3_S9.json
Results image saved to: